## Testing LLM and Spotify API

In [1]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Replace these with your actual Spotify API credentials
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

# Set up the client credentials manager
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Test the connection by retrieving data, e.g., information about a specific artist
artist_name = "Taylor Swift"
results = sp.search(q=artist_name, type='artist')

# Display the artist's information
artist = results['artists']['items'][0]
print(f"Name: {artist['name']}")
print(f"Genres: {artist['genres']}")
print(f"Followers: {artist['followers']['total']}")
print(f"Spotify URL: {artist['external_urls']['spotify']}")

Name: Taylor Swift
Genres: ['pop']
Followers: 124740363
Spotify URL: https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02


In [16]:
#Langchain model
from transformers import pipeline
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="ibm-granite/granite-3.0-3b-a800m-instruct",  # replace with the model name you want
    huggingfacehub_api_token=os.getenv('HUGGING_FACE_TOKEN')
)

# Generate text with the LangChain LLM
response = llm("Recommend me a song from Metallica. But include in your response only the name of the song, and only one song.")
#print(response)

# Process the response to extract only the song name
# Assuming the format is like: '"Out of Love" is a song by Avicii.'
start_quote_index = response.find('"')  # Find the index of the first quote
end_quote_index = response.find('"', start_quote_index + 1)  # Find the index of the closing quote

if start_quote_index != -1 and end_quote_index != -1:
    # Extract the song name using slicing
    song_name = response[start_quote_index + 1:end_quote_index]
    print("Extracted song name:", song_name)
else:
    print("Song name not found.")

Extracted song name: Enter Sandman


In [17]:
# Search for the song
results = sp.search(q=song_name, type='track', limit=1)

# Extract track information
if results['tracks']['items']:
    track = results['tracks']['items'][0]
    print(f"Found track: {track['name']} by {track['artists'][0]['name']}")
else:
    print("Song not found.")

Found track: Enter Sandman (Remastered) by Metallica


## Using promt template

In [36]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

song_description = 'Instrumental music with a piano in the background'

#Templates
song_template = PromptTemplate(
    input_variables=['description'],
    template='Recommend me a song that matches this description: {song_description}.\
        Include in your response only the name of the song, and only one song.'
)

#Chains
song_chain = LLMChain(llm= llm, prompt= song_template)
if song_description:
    response = song_chain.run(song_description)
    
# Process the response to extract only the song name
# Assuming the format is like: '"Out of Love" is a song by Avicii.'
start_quote_index = response.find('"')  # Find the index of the first quote
end_quote_index = response.find('"', start_quote_index + 1)  # Find the index of the closing quote

if start_quote_index != -1 and end_quote_index != -1:
    # Extract the song name using slicing
    song_name = response[start_quote_index + 1:end_quote_index]
    print("Extracted song name:", song_name)
else:
    print("Song name not found.")

Extracted song name: Clair de Lune


In [37]:
# Search for the song
results = sp.search(q=song_name, type='track', limit=1)

# Extract track information
if results['tracks']['items']:
    track = results['tracks']['items'][0]
    print(f"Found track: {track['name']} by {track['artists'][0]['name']}")
else:
    print("Song not found.")

Found track: Clair de lune by Claude Debussy
